<a href="https://colab.research.google.com/github/NUROISEA/anime-webui-colab/blob/main/waifu_diffusion_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>🌟 <code>Waifu Diffusion Colab</code></h2>

- 🤔 [How to use](https://github.com/NUROISEA/anime-webui-colab#-how-to-use)

In [ ]:
#@title 🚀 `Launch web UI`

#@markdown 🧐 Select what Waifu Diffusion model you want to use

model_version = "1.4" #@param ["1.3", "1.3.5", "1.4"]

#@markdown Click the play button on the left to install the web UI and it's dependencies

#@markdown It takes around 3-5 minutes before you can see a `gradio.app` link to the web UI

try:
  has_run
except NameError:
  has_run = False

web_ui_folder = "/content/stable-diffusion-webui"
extensions_folder = f"{web_ui_folder}/extensions"
models_folder = f"{web_ui_folder}/models/Stable-diffusion"
vae_folder = f"{web_ui_folder}/models/VAE"

model_dictionary = {
  "1.3":"https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt",
  "1.3.5":"https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/models/wd-1-3-5_80000-fp32.ckpt",
  "1.4":"https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.ckpt",
}

vae_link = "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt"
vae_name = vae_link.split("/")[-1]

model_link = model_dictionary[model_version]
model_name = model_link.split("/")[-1]

git_flags = "-q"
wget_flags = "-q --show-progress"

if not has_run:
  has_run = True

  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl

  !git clone {git_flags} https://github.com/camenduru/stable-diffusion-webui
  !git clone {git_flags} https://github.com/yfszzx/stable-diffusion-webui-images-browser {extensions_folder}/stable-diffusion-webui-images-browser
  !git clone {git_flags} https://github.com/DominikDoom/a1111-sd-webui-tagcomplete {extensions_folder}/a1111-sd-webui-tagcomplete

  if model_version == "1.4":
    !wget {wget_flags} https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.yaml -O {models_folder}/wd-1-4-anime_e1.yaml
    !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/wd14_ui_config.json -O {web_ui_folder}/wd14_ui_config.json

  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/wd_config.json -O {web_ui_folder}/wd_config.json
  
  !wget {wget_flags} {vae_link} -O {vae_folder}/{vae_name}

try:
  models_downloaded
except NameError:
  models_downloaded = []

if model_version not in models_downloaded:
  models_downloaded.append(model_version)
  !wget {wget_flags} {model_link} -O {models_folder}/{model_name}

arg_list = [
  "--share --xformers",
  f"--ckpt {models_folder}/{model_name}",
  f"--vae-path {vae_folder}/{vae_name}",
  f"--ui-settings-file {web_ui_folder}/wd_config.json"
]

if model_version == "1.4":
  arg_list.append(f"--ui-config-file {web_ui_folder}/wd14_ui_config.json")

%cd {web_ui_folder}
args = " ".join(arg_list)
!python launch.py {args}

In [ ]:
#@title 📩 `Saving images`

#@markdown The zip file will be visible at the files tab.

from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = f"outputs-{str_date}.zip"

%cd /content/stable-diffusion-webui
print("Zipping...")
!zip -qr /content/{archive_name} outputs
print(f"Zipped. You can now find {archive_name} at the files tab.")


#@markdown 📤 Upload zip to [temp.sh](https://temp.sh/)?
upload_to_tempsh = False #@param {type:'boolean'}
if upload_to_tempsh:
  !curl -T /content/{archive_name} temp.sh
  print("\n^^^ This is your download link! ^^^")

#@markdown ----

#@markdown 📲 Copy zip to Google Drive?
copy_to_gdrive = False #@param {type:'boolean'}
#@markdown 📂 The folder path to copy the zip file, if the above is checked, if left blank it will be in `AI/Generated`
gdrive_folder = "AI/Generated" #@param { 'type': 'string' }

if copy_to_gdrive:
  from google.colab import drive

  print("Mounting to Google Drive...")
  drive.mount('/content/drive')
  if gdrive_folder == "":
    gdrive_folder = "AI/Generated"
  
  drive_folder = f"/content/drive/MyDrive/{gdrive_folder}"
  
  !mkdir -p {drive_folder}
  !cp /content/{archive_name} {drive_folder}
  print(f"Copied to {gdrive_folder}!")

#@markdown ----

#@markdown ❌ Delete the files in the colab? (this wont delete the saved zip)
delete_local_files = True #@param {type:'boolean'}
if delete_local_files:
  %cd /content/stable-diffusion-webui
  !rm -rf outputs